## Validate Kafka Connect to produce messages

Let's run Kafka Connect to produce messages into the Kafka Topic.

* Validate whether we have Kafka topic created with our user name prefix.

```shell
kafka-topics.sh \
  --zookeeper m01.itversity.com:2181,m02.itversity.com:2181,r01.itversity.com:2181 \
  --list \
  --topic `whoami`_retail
  
kafka-topics.sh \
  --zookeeper m01.itversity.com:2181,m02.itversity.com:2181,r01.itversity.com:2181 \
  --describe \
  --topic `whoami`_retail
```

* Open the terminal and run the below command to ensure you are in the right directory. 

```shell
cd ~/kafka_connect/retail_logs_produce
```

* Make sure we have created the required properties files to run Kafka connect process to produce messages from the log files under **/opt/gen_logs/logs/access.log** to the Kafka topic `whoami`_retail.

```shell
ls -ltr ~/kafka_connect/retail_logs_produce
```

* Review **retail_logs_standalone.properties** 
  * Bootstrap servers with more than one brokers on multinode cluster.
  * Key and Value Converters
  * Offset file name
  * Make sure **offset.storage.file.filename** is pointing to **/home/YOUR_USER_NAME**. If it is **/home/itversity** replace itversity with your OS account name. You can validate whether the path is correct or not by running `ls -ltr /home/YOUR_USER_NAME/kafka_connect/retail_logs_produce`.

```shell
bootstrap.servers=w01.itversity.com:9092,w02.itversity.com:9092

key.converter=org.apache.kafka.connect.storage.StringConverter
value.converter=org.apache.kafka.connect.storage.StringConverter

key.converter.schemas.enable=true
value.converter.schemas.enable=true

offset.storage.file.filename=/home/itversity/kafka_connect/retail_logs_produce/retail.offsets
# Flush much faster than normal, which is useful for testing/debugging
offset.flush.interval.ms=10000
```

* Review **retail_logs_file_source.properties**
  * Location of the file
  * Topic Name should be based upon your username. Replace **itversity** with your OS account name.

```shell
name=local-file-source
connector.class=FileStreamSource
tasks.max=1
file=/opt/gen_logs/logs/access.log
topic=itversity_retail
```

* We need to make sure that **/opt/gen_logs/logs/access.log** is not too big. In actual environments the log files will be rotated at regular intervals.
* Run the below commands to reset the **/opt/gen_logs/logs/access.log**, if it is too big.

```shell
stop_logs.sh
cat /dev/null > /opt/gen_logs/logs/access.log
start_logs.sh
tail_logs.sh
```

* Run the **connect-standalone.sh** to produce the data from log file as messags into Kafka topic by passing both the properties files.

```shell
connect-standalone.sh \
    retail_logs_standalone.properties \
    retail_logs_file_source.properties
```

* Validate by running `kafka-console-consumer` script in another terminal session.

```shell
kafka-console-consumer.sh \
    --bootstrap-server w01.itversity.com:9092,w02.itversity.com:9092 \
    --topic `whoami`_retail \
    --from-beginning
```